# 92. 平行移動等変性と不変性：CNNの幾何学的性質

## 学習目標

このノートブックでは、以下を学びます：

1. **等変性（Equivariance）**と**不変性（Invariance）**の違い
2. **畳み込み層の等変性**の数学的証明
3. **プーリングによる不変性**の獲得
4. **回転・スケール変換**に対する性質

## 目次

1. [等変性 vs 不変性](#section1)
2. [畳み込みの平行移動等変性](#section2)
3. [プーリングと不変性](#section3)
4. [CNNの全体的な性質](#section4)
5. [他の変換に対する性質](#section5)
6. [まとめ](#summary)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyArrowPatch
import japanize_matplotlib

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

import torch
import torch.nn as nn
import torch.nn.functional as F

<a id="section1"></a>
## 1. 等変性 vs 不変性

### 定義

変換 $T$ と関数 $f$ に対して：

**等変性（Equivariance）**：
$$f(T(x)) = T(f(x))$$
> 入力を変換してから処理 = 処理してから出力を変換

**不変性（Invariance）**：
$$f(T(x)) = f(x)$$
> 入力を変換しても、出力は変わらない

In [ ]:
def visualize_equivariance_vs_invariance():
    """等変性と不変性の違いを可視化"""
    fig, axes = plt.subplots(2, 4, figsize=(18, 10))
    
    # サンプル画像
    img = np.zeros((8, 8))
    img[2:5, 2:5] = 1  # 3x3の正方形
    
    # シフトした画像
    img_shifted = np.zeros((8, 8))
    img_shifted[4:7, 4:7] = 1
    
    # ========== 上段：等変性 ==========
    axes[0, 0].imshow(img, cmap='Blues', vmin=0, vmax=1)
    axes[0, 0].set_title('入力 x', fontsize=12)
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(img_shifted, cmap='Blues', vmin=0, vmax=1)
    axes[0, 1].set_title('T(x)\n(+2, +2シフト)', fontsize=12)
    axes[0, 1].axis('off')
    
    # 畳み込み（エッジ検出）
    kernel = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]], dtype=float)
    
    def simple_conv(img, kernel):
        from scipy.signal import correlate2d
        return correlate2d(img, kernel, mode='same')
    
    out1 = simple_conv(img, kernel)
    out2 = simple_conv(img_shifted, kernel)
    
    axes[0, 2].imshow(out1, cmap='RdBu')
    axes[0, 2].set_title('f(x)\n畳み込み結果', fontsize=12)
    axes[0, 2].axis('off')
    
    axes[0, 3].imshow(out2, cmap='RdBu')
    axes[0, 3].set_title('f(T(x)) = T(f(x))\n同じだけシフト', fontsize=12)
    axes[0, 3].axis('off')
    
    # 等変性の説明
    axes[0, 0].text(4, -1.5, '等変性：f(T(x)) = T(f(x))', fontsize=14, 
                   fontweight='bold', ha='center', color='blue')
    
    # ========== 下段：不変性 ==========
    axes[1, 0].imshow(img, cmap='Blues', vmin=0, vmax=1)
    axes[1, 0].set_title('入力 x', fontsize=12)
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(img_shifted, cmap='Blues', vmin=0, vmax=1)
    axes[1, 1].set_title('T(x)\n(+2, +2シフト)', fontsize=12)
    axes[1, 1].axis('off')
    
    # Global Average Pooling（不変な関数の例）
    gap1 = img.mean()
    gap2 = img_shifted.mean()
    
    # スカラー値を可視化
    axes[1, 2].bar([0], [gap1], color='steelblue', width=0.5)
    axes[1, 2].set_title(f'f(x) = GAP\n= {gap1:.3f}', fontsize=12)
    axes[1, 2].set_ylim(0, 0.2)
    axes[1, 2].set_xticks([])
    
    axes[1, 3].bar([0], [gap2], color='steelblue', width=0.5)
    axes[1, 3].set_title(f'f(T(x)) = GAP\n= {gap2:.3f}', fontsize=12)
    axes[1, 3].set_ylim(0, 0.2)
    axes[1, 3].set_xticks([])
    
    # 不変性の説明
    axes[1, 0].text(4, -1.5, '不変性：f(T(x)) = f(x)', fontsize=14, 
                   fontweight='bold', ha='center', color='red')
    
    plt.suptitle('等変性（Equivariance） vs 不変性（Invariance）', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("等変性：入力のシフトに対して、出力も同じだけシフト（畳み込み）")
    print("不変性：入力のシフトに対して、出力は変化しない（Global Pooling）")

visualize_equivariance_vs_invariance()

### 重要なポイント

- **畳み込み層は等変**：位置情報を保持
- **Global Poolingは不変**：位置情報を捨てる
- **画像分類の最終目標は不変性**：「猫」はどこにいても「猫」

<a id="section2"></a>
## 2. 畳み込みの平行移動等変性

### 数学的証明

平行移動演算子 $T_{\Delta}$ を以下で定義：
$$T_{\Delta}[f](x, y) = f(x - \Delta_x, y - \Delta_y)$$

畳み込み演算子を $*$ とすると：

$$T_{\Delta}[f * w] = T_{\Delta}[f] * w$$

これを証明します。

In [ ]:
def prove_translation_equivariance():
    """平行移動等変性の証明（数式）"""
    print("="*70)
    print("畳み込みの平行移動等変性の証明")
    print("="*70)
    
    print("""
【定義】
- 平行移動演算子: T_Δ[f](x,y) = f(x - Δx, y - Δy)
- 畳み込み: (f * w)(x,y) = Σ_m Σ_n f(x+m, y+n) · w(m, n)

【証明】
左辺: T_Δ[f * w](x, y)
    = (f * w)(x - Δx, y - Δy)
    = Σ_m Σ_n f(x - Δx + m, y - Δy + n) · w(m, n)

右辺: (T_Δ[f] * w)(x, y)
    = Σ_m Σ_n T_Δ[f](x + m, y + n) · w(m, n)
    = Σ_m Σ_n f(x + m - Δx, y + n - Δy) · w(m, n)
    = Σ_m Σ_n f(x - Δx + m, y - Δy + n) · w(m, n)

左辺 = 右辺 ∴ T_Δ[f * w] = T_Δ[f] * w  □

【直感的解釈】
- 畳み込みは「重み共有」のため、どの位置でも同じカーネルを使う
- 入力がシフトしても、各位置での計算は同じ
- 結果として、出力も同じだけシフトする
    """)

prove_translation_equivariance()

In [ ]:
def verify_equivariance_numerically():
    """等変性を数値的に検証"""
    print("="*70)
    print("平行移動等変性の数値検証")
    print("="*70)
    
    # PyTorchで検証
    torch.manual_seed(42)
    
    # 畳み込み層
    conv = nn.Conv2d(1, 1, kernel_size=3, padding=1, bias=False)
    
    # 入力画像
    x = torch.zeros(1, 1, 10, 10)
    x[0, 0, 2:5, 2:5] = torch.randn(3, 3)
    
    # シフトした入力
    shift = 3
    x_shifted = torch.zeros(1, 1, 10, 10)
    x_shifted[0, 0, 2+shift:5+shift, 2+shift:5+shift] = x[0, 0, 2:5, 2:5]
    
    # 畳み込み
    with torch.no_grad():
        y = conv(x)
        y_shifted = conv(x_shifted)
    
    # 方法1: f(T(x)) - 入力をシフトしてから畳み込み
    method1 = y_shifted
    
    # 方法2: T(f(x)) - 畳み込んでからシフト
    method2 = torch.zeros_like(y)
    method2[0, 0, shift:, shift:] = y[0, 0, :-shift, :-shift]
    
    # 比較（有効な領域のみ）
    valid_region1 = method1[0, 0, shift:-1, shift:-1]
    valid_region2 = method2[0, 0, shift:-1, shift:-1]
    
    diff = torch.abs(valid_region1 - valid_region2).max().item()
    
    print(f"\nシフト量: {shift}")
    print(f"f(T(x)) と T(f(x)) の最大差: {diff:.10f}")
    print(f"→ {'等変性が成立！' if diff < 1e-6 else '等変性が成立しない'}")
    
    # 可視化
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    axes[0, 0].imshow(x[0, 0].numpy(), cmap='viridis')
    axes[0, 0].set_title('入力 x', fontsize=12)
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(x_shifted[0, 0].numpy(), cmap='viridis')
    axes[0, 1].set_title(f'T(x) (シフト={shift})', fontsize=12)
    axes[0, 1].axis('off')
    
    axes[0, 2].imshow(conv.weight[0, 0].detach().numpy(), cmap='RdBu')
    axes[0, 2].set_title('カーネル w', fontsize=12)
    axes[0, 2].axis('off')
    
    axes[1, 0].imshow(y[0, 0].detach().numpy(), cmap='RdBu')
    axes[1, 0].set_title('f(x)', fontsize=12)
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(method1[0, 0].detach().numpy(), cmap='RdBu')
    axes[1, 1].set_title('f(T(x))', fontsize=12)
    axes[1, 1].axis('off')
    
    axes[1, 2].imshow(method2[0, 0].detach().numpy(), cmap='RdBu')
    axes[1, 2].set_title('T(f(x))', fontsize=12)
    axes[1, 2].axis('off')
    
    plt.suptitle(f'平行移動等変性の検証: f(T(x)) ≈ T(f(x))\n最大差: {diff:.2e}', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

verify_equivariance_numerically()

<a id="section3"></a>
## 3. プーリングと不変性

### プーリングによる不変性の獲得

プーリングは**局所的な不変性**を与えます。

In [ ]:
def demonstrate_pooling_invariance():
    """プーリングによる不変性のデモ"""
    fig, axes = plt.subplots(2, 4, figsize=(18, 9))
    
    # 4x4の入力で、異なる位置にピークを持つ
    inputs = []
    
    # パターン1: 左上にピーク
    x1 = np.array([[9, 1, 2, 1],
                   [1, 1, 1, 1],
                   [1, 1, 1, 1],
                   [1, 1, 1, 1]], dtype=float)
    
    # パターン2: 右上にピーク（同じ2x2プーリング領域内）
    x2 = np.array([[1, 9, 2, 1],
                   [1, 1, 1, 1],
                   [1, 1, 1, 1],
                   [1, 1, 1, 1]], dtype=float)
    
    # パターン3: 左下にピーク（同じ領域内）
    x3 = np.array([[1, 1, 2, 1],
                   [9, 1, 1, 1],
                   [1, 1, 1, 1],
                   [1, 1, 1, 1]], dtype=float)
    
    # パターン4: 右下にピーク（同じ領域内）
    x4 = np.array([[1, 1, 2, 1],
                   [1, 9, 1, 1],
                   [1, 1, 1, 1],
                   [1, 1, 1, 1]], dtype=float)
    
    inputs = [x1, x2, x3, x4]
    titles = ['(0,0)', '(0,1)', '(1,0)', '(1,1)']
    
    # Max Pooling 2x2
    def max_pool_2x2(x):
        h, w = x.shape
        return np.array([[x[i:i+2, j:j+2].max() 
                         for j in range(0, w, 2)] 
                         for i in range(0, h, 2)])
    
    for idx, (x, title) in enumerate(zip(inputs, titles)):
        # 入力
        axes[0, idx].imshow(x, cmap='Blues', vmin=0, vmax=10)
        for i in range(4):
            for j in range(4):
                axes[0, idx].text(j, i, f'{x[i,j]:.0f}', ha='center', va='center', fontsize=12)
        axes[0, idx].axhline(y=1.5, color='red', linewidth=2, linestyle='--')
        axes[0, idx].axvline(x=1.5, color='red', linewidth=2, linestyle='--')
        axes[0, idx].set_title(f'ピーク位置: {title}', fontsize=12)
        axes[0, idx].axis('off')
        
        # Max Pooling結果
        pooled = max_pool_2x2(x)
        axes[1, idx].imshow(pooled, cmap='Blues', vmin=0, vmax=10)
        for i in range(2):
            for j in range(2):
                axes[1, idx].text(j, i, f'{pooled[i,j]:.0f}', ha='center', va='center', fontsize=14)
        axes[1, idx].set_title(f'MaxPool結果', fontsize=12)
        axes[1, idx].axis('off')
    
    axes[0, 0].text(-0.5, 2, '入力', fontsize=12, ha='right', va='center', fontweight='bold')
    axes[1, 0].text(-0.5, 1, 'MaxPool\n2×2', fontsize=12, ha='right', va='center', fontweight='bold')
    
    plt.suptitle('Max Poolingによる局所的不変性\n左上2×2領域内のどこにピークがあっても、出力は同じ', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("観察：")
    print("- ピークが2×2領域内のどの位置にあっても、MaxPool後の左上の値は9")
    print("- これが『局所的な平行移動不変性』")
    print("- プーリングサイズ内での微小なシフトに対して出力は不変")

demonstrate_pooling_invariance()

In [ ]:
def visualize_global_vs_local_invariance():
    """局所的不変性とグローバル不変性の比較"""
    fig, axes = plt.subplots(1, 3, figsize=(16, 5))
    
    # 入力画像のバリエーション
    np.random.seed(42)
    base_pattern = np.array([[1, 1], [1, 0]])
    
    shifts = [(0, 0), (0, 3), (3, 3)]
    images = []
    
    for dy, dx in shifts:
        img = np.zeros((8, 8))
        img[dy:dy+2, dx:dx+2] = base_pattern
        images.append(img)
    
    # 各画像を表示
    for idx, (img, (dy, dx)) in enumerate(zip(images, shifts)):
        ax = axes[idx]
        ax.imshow(img, cmap='Blues', vmin=0, vmax=1)
        ax.set_title(f'シフト: ({dy}, {dx})', fontsize=12)
        ax.axis('off')
        
        # GAP値
        gap = img.mean()
        ax.text(4, 8.5, f'GAP = {gap:.3f}', fontsize=12, ha='center',
               bbox=dict(boxstyle='round', facecolor='lightyellow'))
    
    plt.suptitle('Global Average Pooling：完全な平行移動不変性\n位置に関係なく同じ出力', 
                fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

visualize_global_vs_local_invariance()

<a id="section4"></a>
## 4. CNNの全体的な性質

### 層ごとの性質の変化

CNN全体を見ると、等変性と不変性が段階的に変化します。

In [ ]:
def visualize_cnn_equivariance_to_invariance():
    """CNNにおける等変性→不変性の遷移"""
    fig, ax = plt.subplots(figsize=(16, 8))
    
    # 層の構成
    layers = [
        ('入力', '等変性:\n100%', 'lightblue'),
        ('Conv1', '等変性:\n100%', 'lightblue'),
        ('Pool1', '局所不変性\n(2×2)', 'lightyellow'),
        ('Conv2', '等変性:\n100%', 'lightblue'),
        ('Pool2', '局所不変性\n(4×4累積)', 'lightyellow'),
        ('Conv3', '等変性:\n100%', 'lightblue'),
        ('Pool3', '局所不変性\n(8×8累積)', 'lightyellow'),
        ('GAP', 'グローバル\n不変性', 'lightgreen'),
        ('FC', '不変性:\n100%', 'lightgreen'),
    ]
    
    # 層を描画
    x_positions = np.linspace(0.05, 0.95, len(layers))
    
    for x, (name, prop, color) in zip(x_positions, layers):
        # ボックス
        from matplotlib.patches import FancyBboxPatch
        box = FancyBboxPatch((x - 0.04, 0.35), 0.08, 0.3,
                             boxstyle="round,pad=0.01",
                             facecolor=color, edgecolor='gray',
                             transform=ax.transAxes)
        ax.add_patch(box)
        
        # 層名
        ax.text(x, 0.72, name, ha='center', va='center', fontsize=11,
               fontweight='bold', transform=ax.transAxes)
        
        # 性質
        ax.text(x, 0.5, prop, ha='center', va='center', fontsize=9,
               transform=ax.transAxes)
    
    # 矢印
    for i in range(len(layers) - 1):
        ax.annotate('', xy=(x_positions[i+1] - 0.05, 0.5),
                   xytext=(x_positions[i] + 0.05, 0.5),
                   arrowprops=dict(arrowstyle='->', color='gray'),
                   transform=ax.transAxes)
    
    # 凡例
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='lightblue', edgecolor='gray', label='畳み込み（等変）'),
        Patch(facecolor='lightyellow', edgecolor='gray', label='プーリング（局所不変）'),
        Patch(facecolor='lightgreen', edgecolor='gray', label='グローバル不変'),
    ]
    ax.legend(handles=legend_elements, loc='lower center', ncol=3, fontsize=11)
    
    # グラデーションバー
    gradient = np.linspace(0, 1, 100).reshape(1, -1)
    ax_gradient = ax.inset_axes([0.1, 0.1, 0.8, 0.05])
    ax_gradient.imshow(gradient, aspect='auto', cmap='coolwarm')
    ax_gradient.set_xticks([0, 50, 100])
    ax_gradient.set_xticklabels(['等変性', '', '不変性'])
    ax_gradient.set_yticks([])
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.axis('off')
    ax.set_title('CNNの層を通じた等変性→不変性の遷移', fontsize=16, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

visualize_cnn_equivariance_to_invariance()

In [ ]:
def explain_equivariance_invariance_balance():
    """等変性と不変性のバランスの説明"""
    print("="*70)
    print("CNNにおける等変性と不変性のバランス")
    print("="*70)
    
    print("""
【等変性（畳み込み層）の役割】
- 位置情報を保持
- 「どこに」特徴があるかを伝える
- セマンティックセグメンテーションなどに必要

【不変性（プーリング層）の役割】
- 位置の微小変動を吸収
- ノイズや歪みに対する頑健性
- 分類タスクに必要

【タスクによる使い分け】

┌─────────────────┬──────────────────────────────────┐
│ タスク          │ 必要な性質                        │
├─────────────────┼──────────────────────────────────┤
│ 画像分類        │ 不変性重視（位置は問わない）      │
│ 物体検出        │ 等変性維持（位置を知りたい）      │
│ セグメンテーション│ 等変性維持（ピクセル単位の位置） │
│ 姿勢推定        │ 等変性維持（関節の位置）          │
└─────────────────┴──────────────────────────────────┘
    """)

explain_equivariance_invariance_balance()

<a id="section5"></a>
## 5. 他の変換に対する性質

CNNは平行移動に対しては等変ですが、他の変換に対してはどうでしょうか？

In [ ]:
def test_other_transformations():
    """回転・スケールに対するCNNの性質"""
    from scipy.ndimage import rotate, zoom
    
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    
    # 元画像
    np.random.seed(42)
    img = np.zeros((32, 32))
    img[10:22, 10:22] = 1
    img[14:18, 14:18] = 0
    
    # カーネル
    kernel = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]], dtype=float)
    
    def simple_conv(img, kernel):
        from scipy.signal import correlate2d
        return correlate2d(img, kernel, mode='same')
    
    # ========== 行1: 平行移動（等変） ==========
    axes[0, 0].imshow(img, cmap='Blues')
    axes[0, 0].set_title('元画像', fontsize=11)
    axes[0, 0].axis('off')
    
    img_translated = np.roll(np.roll(img, 5, axis=0), 5, axis=1)
    axes[0, 1].imshow(img_translated, cmap='Blues')
    axes[0, 1].set_title('平行移動 (+5, +5)', fontsize=11)
    axes[0, 1].axis('off')
    
    out_orig = simple_conv(img, kernel)
    out_trans = simple_conv(img_translated, kernel)
    
    axes[0, 2].imshow(out_orig, cmap='RdBu')
    axes[0, 2].set_title('Conv(元)', fontsize=11)
    axes[0, 2].axis('off')
    
    axes[0, 3].imshow(out_trans, cmap='RdBu')
    axes[0, 3].set_title('Conv(平行移動)\n→ 等変 ✓', fontsize=11)
    axes[0, 3].axis('off')
    
    # ========== 行2: 回転（非等変） ==========
    axes[1, 0].imshow(img, cmap='Blues')
    axes[1, 0].set_title('元画像', fontsize=11)
    axes[1, 0].axis('off')
    
    img_rotated = rotate(img, 45, reshape=False)
    axes[1, 1].imshow(img_rotated, cmap='Blues')
    axes[1, 1].set_title('回転 (45°)', fontsize=11)
    axes[1, 1].axis('off')
    
    out_rot = simple_conv(img_rotated, kernel)
    out_orig_rot = rotate(out_orig, 45, reshape=False)
    
    axes[1, 2].imshow(out_rot, cmap='RdBu')
    axes[1, 2].set_title('Conv(回転)', fontsize=11)
    axes[1, 2].axis('off')
    
    axes[1, 3].imshow(out_orig_rot, cmap='RdBu')
    axes[1, 3].set_title('回転(Conv(元))\n→ 非等変 ✗', fontsize=11)
    axes[1, 3].axis('off')
    
    # ========== 行3: スケール（非等変） ==========
    axes[2, 0].imshow(img, cmap='Blues')
    axes[2, 0].set_title('元画像', fontsize=11)
    axes[2, 0].axis('off')
    
    img_scaled = zoom(img, 1.5)[4:36, 4:36]  # 中心を切り出し
    axes[2, 1].imshow(img_scaled, cmap='Blues')
    axes[2, 1].set_title('スケール (1.5×)', fontsize=11)
    axes[2, 1].axis('off')
    
    out_scaled = simple_conv(img_scaled, kernel)
    
    axes[2, 2].imshow(out_scaled, cmap='RdBu')
    axes[2, 2].set_title('Conv(スケール)', fontsize=11)
    axes[2, 2].axis('off')
    
    axes[2, 3].text(0.5, 0.5, 'スケールにも\n非等変 ✗', fontsize=14, 
                   ha='center', va='center', transform=axes[2, 3].transAxes)
    axes[2, 3].axis('off')
    
    plt.suptitle('CNNの変換に対する性質', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("まとめ:")
    print("- 平行移動: 等変（重み共有のため）")
    print("- 回転: 非等変（データ拡張で対処）")
    print("- スケール: 非等変（マルチスケール処理で対処）")

test_other_transformations()

<a id="summary"></a>
## 6. まとめ

### 学んだこと

1. **等変性 vs 不変性**
   - 等変性: f(T(x)) = T(f(x)) - 変換が出力に伝播
   - 不変性: f(T(x)) = f(x) - 変換しても出力は同じ

2. **畳み込みの性質**
   - 平行移動に対して等変（重み共有による）
   - 回転・スケールには非等変

3. **プーリングの役割**
   - 局所的な不変性を付与
   - 累積的に不変性の範囲が拡大

4. **CNNの全体設計**
   - 浅い層: 等変性を維持（位置情報を保持）
   - 深い層: 不変性を獲得（分類に必要）

### 次のノートブック

次のノートブックでは、**CNNとMLPの比較**を通じて、帰納バイアスの効果をより深く理解します。